# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [8]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

gen_list = [sqrt_value for sqrt_value in generator]

for sqrt_value in gen_list:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [24]:
# Question 1: What is the sum of the outputs of the generator for limit = 5?
sum(gen_list)

8.382332347441762

In [26]:
# Question 2: What is the 13th number yielded
print([i for i in square_root_generator(13)][-1])

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [43]:
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework_starter')

info = generators_pipeline.run(
    people_1(),
    table_name="people",
    write_disposition="replace"
)
print(info)



# Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n People table below: ")
people_data = conn.sql("SELECT SUM(age) FROM people").df()
display(people_data)
conn.close()










def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework_starter')

info = generators_pipeline.run(
    people_2(),
    table_name="people",
    write_disposition="append"
)
print(info)


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n People table below: ")
people_data = conn.sql("SELECT SUM(age) FROM people").df()
display(people_data)
conn.close()

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
Pipeline dlt_ipykernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework_starter
The duckdb destination used duckdb:///D:\ProjectFiles\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708198378.8388429 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 People table below: 


,sum(age)
0,140.0


{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework_starter
The duckdb destination used duckdb:///D:\ProjectFiles\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708198379.650236 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 People table below: 


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [45]:
%%capture
!pip install dlt[duckdb]

In [50]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework_starter')

info = generators_pipeline.run(
    people_1(),
    table_name="people_3",
    write_disposition="merge",
    primary_key="id"
)
print(info)

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework_starter')

info = generators_pipeline.run(
    people_2(),
    table_name="people_3",
    write_disposition="merge",
    primary_key="id"
)
print(info)


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n People table below: ")
people_data = conn.sql("SELECT SUM(age) FROM people_3").df()
display(people_data)
conn.close()

2024-02-18 01:43:54,068|[WARNING              ]|16844|13628|dlt|reference.py|_verify_schema:357|A column record_hash in table people_2 in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-18 01:43:54,159|[WARNING              ]|16844|13628|dlt|reference.py|_verify_schema:357|A column record_hash in table people_2 in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


Pipeline dlt_ipykernel_launcher load step completed in 0.93 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework_starter
The duckdb destination used duckdb:///D:\ProjectFiles\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708199033.611483 is LOADED and contains no failed jobs


2024-02-18 01:43:55,546|[WARNING              ]|16844|13628|dlt|reference.py|_verify_schema:357|A column record_hash in table people_2 in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-18 01:43:55,645|[WARNING              ]|16844|13628|dlt|reference.py|_verify_schema:357|A column record_hash in table people_2 in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


Pipeline dlt_ipykernel_launcher load step completed in 0.91 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework_starter
The duckdb destination used duckdb:///D:\ProjectFiles\data-engineering-zoomcamp\cohorts\2024\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708199035.0674777 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_2            │
│ people_3            │
└─────────────────────┘




 People table below: 


,sum(age)
0,266.0


In [51]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))
print("\n\n\n People table below: ")
people_data = conn.sql("SELECT * FROM people_3").df()
display(people_data)
conn.close()

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_2            │
│ people_3            │
└─────────────────────┘




 People table below: 


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1708199033.611483,4WW9Dhv8OgnOjA,None
1,1,Person_1,26,City_A,1708199033.611483,oxg3o7YqlALUAg,None
2,5,Person_5,35,City_B,1708199035.0674777,pUAnJ/tzSn/Vtw,Job_5
3,7,Person_7,37,City_B,1708199035.0674777,3ZadyAzWdEymiQ,Job_7
4,3,Person_3,33,City_B,1708199035.0674777,KsRSHizWC6z49A,Job_3
5,6,Person_6,36,City_B,1708199035.0674777,5Kp7H1l7W6CAkg,Job_6
6,4,Person_4,34,City_B,1708199035.0674777,dl7tYCNtqlmHUg,Job_4
7,8,Person_8,38,City_B,1708199035.0674777,BQsCmTv8OZqKVg,Job_8


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX